In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

In [11]:
data = pd.read_csv("data.csv")

In [12]:
data.describe()

,Вектор Мощи,Скорость перехода через портал,Приток Силы Потока,Ритм магического ядра,Поток Энергий,Сила Левого Потока,Сила Правого Потока,Пламя Стихий,Температура вдоха Истока,Приток давления Выдоха Истока,Давление вдоха Истока,Давление выдоха Истока,Печать Чародея,Эмульсия Истока,Дыхание Истока,Гармония Бессмертия
count,11934.000000,11934.000000,11934.000000,11934.000000,11934.000000,11934.000000,11934.000000,11934.000000,11934.0,11934.000000,1.193400e+04,11934.000000,11934.000000,11934.000000,11934.00000,11934.0000
mean,5.166667,7.716600,27247.498685,223.711688,8200.947312,227.335768,227.335768,735.495446,736.0,2.352963,1.394603e+00,12.297123,33.641261,0.662440,0.97500,0.9875
std,2.626388,3.985002,22148.613155,81.061874,1091.315507,200.495889,200.495889,173.680552,0.0,1.084770,2.220539e-16,5.337448,25.841363,0.507132,0.01472,0.0075
min,1.138000,1.543320,253.547000,136.939406,6589.002000,5.304000,5.304000,442.364000,736.0,1.093000,1.394603e+00,5.828000,0.000000,0.068000,0.95000,0.9750
25%,3.144000,4.629960,8375.883750,145.220958,7058.324000,60.317000,60.317000,589.872750,736.0,1.389000,1.394603e+00,7.447250,13.677500,0.246000,0.96200,0.9810
50%,5.140000,7.716600,21630.659000,201.514947,8482.081500,175.268000,175.268000,706.038000,736.0,2.083000,1.394603e+00,11.092000,25.276500,0.496000,0.97500,0.9875
75%,7.148000,10.803240,39001.426750,280.447777,9132.606000,332.364750,332.364750,834.066250,736.0,2.981000,1.394603e+00,15.658000,44.552500,0.882000,0.98800,0.9940
max,9.300000,13.889880,72784.872000,372.879926,9797.103000,645.249000,645.249000,1115.797000,736.0,4.560000,1.394603e+00,23.140000,92.556000,1.832000,1.00000,1.0000
